# Convolutional Neural Networks

In [ ]:
%matplotlib inline
import warnings
import numpy as np
import pandas as pd
import tarfile
from skimage.io import imread, imshow
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical

warnings.filterwarnings("ignore")

In [ ]:
# utility functions
def zero_pad(X, pad):
    return np.pad(X, ((pad, pad), (pad, pad), (0, 0)), 'constant')

def load_data():

    with tarfile.open('../data/data.tar.gz', 'r') as f:
        f.extractall()

    df_train = pd.read_csv('fashion_mnist_train.csv')
    df_test = pd.read_csv('fashion_mnist_test.csv')

    x_train = df_train.drop('label', axis=1).as_matrix().astype(np.uint8)
    y_train = df_train['label'].as_matrix().astype(np.uint8)
    x_test = df_test.drop('label', axis=1).as_matrix().astype(np.uint8)
    y_test = df_test['label'].as_matrix().astype(np.uint8)

    return (x_train, y_train), (x_test, y_test)

img = imread('./logo.png') # the test image we will be using
kernel = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]])
kernel = np.stack([kernel]*3, axis=2)

In [ ]:
imshow(img)

## Convolutional neural net building blocks

### Convolutional layer

In [ ]:
def conv2d(X, W):
    pass

In [ ]:
convolved = conv2d(img, kernel)
imshow(convolved, cmap='gray')

### Padding and strides

**Padding** and **stride** determine the conv layer output size.   

'same' padding means that the output will have the same spatial dimensions as the input. 'valid' padding is equivalent to no padding, i.e. the output will be smaller than the input.   

Stride determines by how much the kernel is moved between feature computations (by how many pixels we 'slide' it). 

In practice, we usually use 'same' padding and stride 1 for the convolutional layers - output has the same spatial dimensions as the input. However, those parameters are important in *pooling layers* (see below).  

Read [this guide](https://arxiv.org/pdf/1603.07285.pdf) for an in-depth look at convolution arithmetics.

In [ ]:
def compute_output_shape(input_shape, kernel_size, pad, stride):
    return ((input_shape[0] - kernel_size + 2 * pad) // stride + 1,
            (input_shape[1] - kernel_size + 2 * pad) // stride + 1)

def conv2d_padding(X, W, padding='same', stride=1):
    pass

In [ ]:
same_1 = conv2d_padding(img, kernel)
valid_2 = conv2d_padding(img, kernel, 'valid', 2)
same_1.shape, valid_2.shape

## Pooling

In [ ]:
def max_pool(X, window_size, padding='valid', stride=2):
    pass

In [ ]:
pooled = max_pool(img, 2)

In [ ]:
pooled.shape

## Dropout

In [ ]:
def dropout(X, p):
    pass

In [ ]:
x = np.random.random((1, 10))
imshow(x > 0, cmap='inferno')

In [ ]:
imshow(dropout(x, .5) > 0, cmap='inferno')

## Full network implementation in Keras

In [ ]:
# load the data
(x_train, y_train), (x_test, y_test) = load_data()
y_train = to_categorical(y_train) # convert to one-hot encoding
y_test = to_categorical(y_test)

### Standard feedforward neural network

In [ ]:
# if the training is too slow
feedforward = load_model('../models/feedforward.h5')

In [ ]:
_, score_ff = feedforward.evaluate(x_test, y_test, batch_size=128)
score_ff

### A simple convnet from scratch

In [ ]:
# if the training is too slow
convnet = load_model('../models/convnet.h5')

In [ ]:
_, score_conv = convnet.evaluate(x_test, y_test, batch_size=128)
score_conv

## Further exercises
- There is an asymptotically faster way of doing convolutions. Implement this method (*note:* use only one channel for simplicity (i.e. no depth).   
*Hint:* there is a transform $T$ such that $T(W * X) = T(W)T(X)$, i.e. convolution is equivalent to multiplication under the transform.
- It can be shown that spatial convolution can be rewritten in terms of matrix multiplication. Try to implement this approach. This does not improve the asymptotical complexity. However, most modern DL libraries (including ```Tensorflow```) use this method. Why?  
*Hint:* read (this)[http://cs231n.github.io/convolutional-networks/#conv] and investigate ```im2col```).
- A frequently used regularization technique, in addition to dropout, is *batch normalization*. It normalizes the activations after each layer so that they follow approximately standard normal distribution. Try experimenting with it in Keras (available as ```keras.layers.BatchNormalization```). How does it improve the training? Read [the original paper](https://arxiv.org/pdf/1502.03167.pdf) for further information.
- Find an interesting classification problem (if you're out of ideas, try the classic [dogs vs cats](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data) problem) and implement *transfer learning* to solve it. Keras makes several pre-trained models available in ```keras.applications``` - try VGG16 first. Test how many training images do you need to acheive good performance.